# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235400202385                   -0.50    8.0         
  2   -7.250146123437       -1.83       -1.40    1.0   9.07ms
  3   -7.251173293030       -2.99       -2.01    2.0   9.93ms
  4   -7.251158019544   +   -4.82       -2.03    2.0   10.4ms
  5   -7.251316454037       -3.80       -2.48    1.0   9.48ms
  6   -7.251338275495       -4.66       -3.21    1.0   9.14ms
  7   -7.251338702607       -6.37       -3.59    1.0   9.31ms
  8   -7.251338783127       -7.09       -4.00    2.0   10.7ms
  9   -7.251338798059       -7.83       -4.70    2.0   10.5ms
 10   -7.251338798606       -9.26       -5.10    2.0   10.4ms
 11   -7.251338798692      -10.06       -5.62    1.0   9.49ms
 12   -7.251338798703      -10.95       -5.92    2.0   11.1ms
 13   -7.251338798704      -11.84       -6.41    1.0   9.41ms
 14   -7.251338798705      -12.70       -6.92    2.0   11.1ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06796513521591432
[ Info: Arnoldi iteration step 2: normres = 0.2477673948144414
[ Info: Arnoldi iteration step 3: normres = 0.3314871776516751
[ Info: Arnoldi iteration step 4: normres = 0.539277225980295
[ Info: Arnoldi iteration step 5: normres = 0.927838292787568
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.01e-01, 1.95e-01, 7.21e-01, 2.27e-01, 1.86e-02)
[ Info: Arnoldi iteration step 6: normres = 0.32657902068402767
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.49e-01, 5.94e-02, 1.30e-01, 1.17e-01, 8.15e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10313002183466759
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.23e-02, 5.58e-03, 1.14e-02, 7.43e-02, 5.56e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13270483369133418
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.18e-04